In [1]:
import numpy as np
import numpy.random as rnd

import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

import time
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#### Все места, где нужно дописать код отмечены TODO.

## Считывание и подготовка данных.

In [4]:
# Считываем данные: каждый класс лежит в своем csv файле. 
male = pd.read_csv('male.csv',header = None)[0]
female = pd.read_csv('female.csv',header = None)[0]

y = np.hstack((np.zeros(len(male)),np.ones(len(female))))
data = list(male)
data.extend(list(female))

In [5]:
# Для дальнейшей работы нам понадобится словарь символов + 
# мы не будем различать строчные и прописные буквы + 
# у нас все последовательности разной длины и нам нужно понимать, какова макимальная длина + 
# нам нужен отдельный символ под паддинг, чтобы уметь работать с последовательностями разной длины
MAX_LEN = 0
chars = set()
for i in xrange(len(data)):
    data[i] = data[i].lower()
    MAX_LEN = max(MAX_LEN,len(data[i]))
    chars = chars.union(set(data[i][:]))
    
chars = list(chars)
PAD_CHAR = '_PADDING_'
chars = [PAD_CHAR] + chars
char_to_id = { ch:id for id,ch in enumerate(chars) }
id_to_char = { id:ch for id,ch in enumerate(chars) }

VOCAB_SIZE = len(chars)

In [6]:
# Разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)

In [7]:
def data2format(data, labels):
    """Функция преобразует выбоку данных в формат, подходящий для подачи в нейронную сеть.
    
    data - список строк (пример - X_train)
    labels - вектор меток для строк из data (пример - y_train)
    
    Дальше за N обозначается число строк в data
    
    Вернуть нужно словарь со следующими элементами:
    x - матрица размера [N, MAX_LEN], в которой каждая строка соответствует строке в data:
        вся строка кодируется с помощью char_to_id, недостающие элементы в конце коротких строк заполняются символом PAD_CHAR
    y - вектор длины N с метками
    
    """
    
    # TODO
    

In [8]:
train_data = data2format(X_train,y_train)
test_data = data2format(X_test,y_test)

## Вспомогательные функции

In [10]:
# Необходимые константы
NUM_EPOCHS = 20
BATCH_SIZE = 100
SEQ_LEN = 20
LEARNING_RATE = 0.01
GRAD_CLIP = 100

In [11]:
# Технические вещи

# Вспомогательная функция для запаковки результата обучения 
def pack(train_err, train_acc, test_err, test_acc, network):
    return {'train_err':train_err, 
        'train_acc':train_acc, 
        'test_err':test_err, 
        'test_acc':test_acc, 
        'network':network
           } 

# устойчивая реализация кросс-энтропии
def BinaryCrossentropy(probs, labels):
    probs = probs[:, 0]
    probs = torch.clamp(probs, 1e-7, 1-1e-7)
    labels = labels.type(probs.data.type())
    return -(labels * torch.log(probs) + (1 - labels) * (torch.log(1 - probs))).mean()

## Нейронная сеть

В простейшем случае мы будем использовать сеть, которая считывает входную последовательность, и выдает результат только в самом конце.

In [12]:
# Для работы с последовательностями разной длины стоит использовать эти функции.
# Обратите внимание, что последователньости нужно отсортировать перед подачей в pack_padded_sequence.
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Net(nn.Module):
    """Класс задает простейшую рекуррентную сеть, которая принимает на вход батч размера [BATCH_SIZE, MAX_LEN] 
    и применяет к нему следующие преобразования:
    
    1. Embedding для перевода кодировки символов в нормальное представление: VOCAB_SIZE -> emb_size
    2. Рекуррентный слой c n_hidden элементов на скрытом слое. Из этого слоя нам нужно только выход в последний момент времени.
    3. Полносвязный слой для бинарной классификации с sigmoid в качестве нелинейности.
    
    * Обратите внимание на параметр batch_first у рекуррентного слоя.
    """
    def __init__(self, embedding_dim, hidden_dim, vocab_size, batch_size):
        # TODO

    def forward(self, names, lengths):
        # TODO

In [17]:
def train(train_data, test_data, emb_size, n_hidden, show = False):
    """Функция обучает нейросеть по данным train_data + контролирует процесс по качеству на test_data
    Следует обратить внимание на следующее:
    1. Сеть будем учить NUM_EPOCHS эпох, в каждой из столько батчей, сколько есть в train_data
    2. Генерировать батчи можно с помощью батчгенератора pytorch. Для этого пригодятся torch.utils.data.TensorDataset
        и torch.utils.data.DataLoader.
    3. Для того, чтобы следить за процессом обучения будем считать средний loss и 
        среднюю точность классификации на всех батчах трейна и теста и сохранять эти данные 
        в соответствующие массивы. 
    4. Перед тем, как делать шаг по градиенту, будем ограничивать градиент по норме значением GRAD_CLIP
    
    """
    print("Prepare data ...")
    # задаем батчгенераторы
    # TODO
    
    train_size = len(train_data['x'])
    test_size = len(test_data['x'])
    num_train_batches = train_size / BATCH_SIZE
    num_test_batches = test_size / BATCH_SIZE
    train_err=np.zeros(NUM_EPOCHS)
    train_acc=np.zeros(NUM_EPOCHS)
    test_err=np.zeros(NUM_EPOCHS)
    test_acc=np.zeros(NUM_EPOCHS)

    print("Building network ...")
    # Строим сеть и переносим ее на cuda, если нужно
    # TODO
    print("The network has {} params".format(sum([x.data.numel() for x in net.parameters()])))
    
    # Задаем оптимизатор, рекомендуется использовать adam
    # TODO
    
    print("Training ...")
    for epoch in xrange(NUM_EPOCHS):
        start_time = time.time()
        # TODO

        print("Epoch {} \t loss / accuracy test = {:.4f}, {:.4f} \t train = {:.4f}, {:.4f} \t time = {:.2f}s".
              format(epoch, test_err[epoch],test_acc[epoch], 
                     train_err[epoch],  train_acc[epoch],time.time() - start_time))
             
    return pack(train_err, train_acc, test_err, test_acc, net)

Перед тем, как запускать обучение большой сети на большое число эпох, проверьте, что маленькая сеть выдает вменяемые результаты: качество больше 50%.

In [18]:
model = train(train_data, test_data, 40, 100)

Prepare data ...
Building network ...
The network has 58101 params
Training ...
Epoch 0 	 loss / accuracy test = 0.0047, 0.7215 	 train = 0.0057, 0.6723 	 time = 0.56s
Epoch 1 	 loss / accuracy test = 0.0042, 0.7563 	 train = 0.0045, 0.7631 	 time = 0.41s
Epoch 2 	 loss / accuracy test = 0.0040, 0.7714 	 train = 0.0042, 0.7879 	 time = 0.43s
Epoch 3 	 loss / accuracy test = 0.0039, 0.7701 	 train = 0.0039, 0.8079 	 time = 0.41s
Epoch 4 	 loss / accuracy test = 0.0038, 0.7798 	 train = 0.0038, 0.8119 	 time = 0.42s
Epoch 5 	 loss / accuracy test = 0.0038, 0.7819 	 train = 0.0036, 0.8182 	 time = 0.41s
Epoch 6 	 loss / accuracy test = 0.0037, 0.7785 	 train = 0.0035, 0.8263 	 time = 0.41s
Epoch 7 	 loss / accuracy test = 0.0036, 0.7836 	 train = 0.0033, 0.8354 	 time = 0.42s
Epoch 8 	 loss / accuracy test = 0.0036, 0.7898 	 train = 0.0032, 0.8439 	 time = 0.40s
Epoch 9 	 loss / accuracy test = 0.0036, 0.7898 	 train = 0.0031, 0.8451 	 time = 0.40s
Epoch 10 	 loss / accuracy test = 0.0035

## Посмотрим что из этого вышло

In [19]:
def predict(name, model):
    """Функция выдает предсказание обученной модели model для имени name.
    Предсказание - число из [0,1] - вероятность того, что имя женское
    """
    # TODO

In [20]:
dataset = set(data)

In [21]:
name = 'Yaroslav'.lower()
if name in dataset:
    print 'This name is in our dataset'
else:
    print 'This is a new name'
pred = predict(name, model)
if pred>=0.5:
    print "It's female name"
else:
    print "It's male name"
print pred

This is a new name
It's male name
0.0199103318155


In [22]:
name = 'Polina'.lower()
if name in dataset:
    print 'This name is in our dataset'
else:
    print 'This is a new name'
pred = predict(name, model)
if pred>=0.5:
    print "It's female name"
else:
    print "It's male name"
print pred

This is a new name
It's female name
0.996361672878


## Дополнительные пункты

1. Обучение более сложной модели и контроль переобучения. Попробуйте подобрать хорошую модель RNN для данной задачи. Для этого проанализируйте качество работы модели в зависимости от ее размеров, попробуйте использовать многослойную сеть. Также нужно проконтролировать переобучение моделей. Для этого можно выделить тестовый кусок из текста и смотреть на то, как меняется loss на нем в процессе обучения. Если на графиках видно переобучение, то стоит добавить dropout слои в модель (обычный dropout до, между и после рекуррентных слоев). При использовании дропаута на стадии предсказания для нового объекта нужно ставить флаг deterministic=True.
2. Другая архитектура 1. Попробуйте использовать не только состоянию скрытых переменных в последний момент времени, а усреднение/максимум значений скрытых переменных во все моменты времени. Попробуйте двунаправленную сеть при таком подходе. 
3. Другая архитектура 2. Попробуйте использовать не только состоянию скрытых переменных в последний момент времени, а сумму значений скрытых переменных во все моменты времени с коэффициентами attention. Попробуйте двунаправленную сеть при таком подходе. Attention коэффициент для определенного момента времени может представлять собой просто линейную комбинацию значений скрытых переменных в этот момент времени с обучаемыми весами.
3. Визуализация. Попробуйте провизуализировать результаты. Например, для стандартной архитектуры можно посмотреть на изменение предсказания во времени: на каких элементах предсказание значительнее всего изменяется в сторону одного или другого класса? При использовании схемы из 2/3 пункта, можно смотреть на вклад каждого момента времени в результат. Так как после рекуррентного слоя у нас стоит просто линейный классификатор, то можно посмотреть, что выдает этот классификатор при применении к скрытым переменным в каждый момент времени. Таким образом выделяться те буквы, которые голосуют за один класс и те, которые голосуют за другой.
4. Batchnorm и Layernorm. Запрограммируйте RNN c layer normalization из статьи [Lei Ba et al., 2016]. Поэкспериментируйте с применением обычной batch normalization и layer normalization, сравните результаты.